In [21]:
import os
from pathlib import PurePath
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [22]:
outputdir = PurePath(os.environ['DATA'], 'STEREO/L7tum/')
csvf = PurePath(outputdir, 'training_set', 'targets.csv')
df = pd.read_csv(csvf, index_col=0)
df

,frame,blob_id_init,blob_id,x,y,xblob,yblob,sig_b,sig_a
0,0,21,21,26.0,110.0,26.0,108.0,20.48,4.096
1,0,52,52,16.0,461.0,16.0,465.0,20.48,4.096
2,0,159,159,53.0,124.0,55.0,117.0,20.48,4.096
3,0,346,346,112.0,107.0,110.0,109.0,20.48,4.096
4,0,369,369,293.0,205.0,293.0,206.0,20.48,4.096
...,...,...,...,...,...,...,...,...,...
195,9,1172,1536,127.0,258.0,127.0,259.0,8.00,1.600
196,9,915,1874,265.0,164.0,265.0,162.0,8.00,1.600
197,9,277,27,477.0,94.0,479.0,83.0,20.48,4.096
198,9,774,607,539.0,509.0,538.0,509.0,12.80,2.560


In [23]:
df.query('blob_id_init == 21')

,frame,blob_id_init,blob_id,x,y,xblob,yblob,sig_b,sig_a
0,0,21,21,26.0,110.0,26.0,108.0,20.48,4.096
20,1,21,27,26.0,112.0,26.0,116.0,20.48,4.096
40,2,21,32,24.0,123.0,26.0,123.0,20.48,4.096
60,3,21,42,24.0,137.0,25.0,130.0,20.48,4.096
80,4,21,59,24.0,138.0,25.0,137.0,20.48,4.096
100,5,21,100,23.0,143.0,25.0,145.0,20.48,4.096
120,6,21,185,23.0,148.0,24.0,151.0,20.48,4.096
140,7,21,358,23.0,156.0,24.0,155.0,12.80,2.560
160,8,21,643,23.0,161.0,24.0,161.0,12.80,2.560
180,9,21,1155,23.0,166.0,23.0,167.0,8.00,1.600


In [24]:
nframes = 10
blob_inits = df[df['frame'] == 0]['blob_id_init']
nblobs = len(blob_inits)
print(nblobs)

20


In [26]:
pos_array = np.zeros((nblobs, nframes, 2))
for f in range(nframes):
    for i, b in enumerate(blob_inits):
        pos_array[i, f, 0] = df[(df['frame'] == f) & (df['blob_id_init']==b)]['xblob']
        pos_array[i, f, 1] = df[(df['frame'] == f) & (df['blob_id_init']==b)]['yblob']

print('(x,y) coordinates of the 1st blob through time')
pos_array[0, :, :]

(x,y) coordinates of the 1st blob through time


array([[ 26., 108.],
       [ 26., 116.],
       [ 26., 123.],
       [ 25., 130.],
       [ 25., 137.],
       [ 25., 145.],
       [ 24., 151.],
       [ 24., 155.],
       [ 24., 161.],
       [ 23., 167.]])

In [27]:
vel_array = pos_array[:, 1:, :] - pos_array[:, 0:-1, :]
print('(vx, vy) of the 1st blob through time')
vel_array[0, :, :]

(vx, vy) of the 1st blob through time


array([[ 0.,  8.],
       [ 0.,  7.],
       [-1.,  7.],
       [ 0.,  7.],
       [ 0.,  8.],
       [-1.,  6.],
       [ 0.,  4.],
       [ 0.,  6.],
       [-1.,  6.]])

In [28]:
avg_vel = pos_array[:, -1, :] - pos_array[:, 0, :]
avg_vel

array([[-3., 59.],
       [-3., 55.],
       [ 2., 73.],
       [-1., 57.],
       [ 0., 63.],
       [-1., 48.],
       [ 2., 61.],
       [-8., 46.],
       [ 0., 55.],
       [ 0., 66.],
       [ 1., 58.],
       [-4., 36.],
       [-2., 47.],
       [ 0., 75.],
       [ 0., 45.],
       [ 0., 53.],
       [-1., 68.],
       [-2., 34.],
       [-1., 53.],
       [-2., 53.]])

### Get physical coordinates

In [29]:
DSUN_OBS= 1.433675e6  # Solar diameter in km
RSUN = DSUN_OBS/2  # Solar radius in km
DT = 5*60 # 5 minutes time interval between each frame
CDELT2  = 0.0141875720451562 # solar radius / px
VUNIT = CDELT2 * RSUN/DT 

In [30]:
avg_vel_km = avg_vel*VUNIT/9
avg_vel_km

array([[-11.30020408, 222.23734699],
       [-11.30020408, 207.17040821],
       [  7.53346939, 274.97163272],
       [ -3.76673469, 214.7038776 ],
       [  0.        , 237.30428577],
       [ -3.76673469, 180.80326535],
       [  7.53346939, 229.77081638],
       [-30.13387756, 173.26979596],
       [  0.        , 207.17040821],
       [  0.        , 248.60448986],
       [  3.76673469, 218.4706123 ],
       [-15.06693878, 135.60244901],
       [ -7.53346939, 177.03653065],
       [  0.        , 282.50510211],
       [  0.        , 169.50306127],
       [  0.        , 199.63693882],
       [ -3.76673469, 256.13795925],
       [ -7.53346939, 128.06897962],
       [ -3.76673469, 199.63693882],
       [ -7.53346939, 199.63693882]])

In [31]:
avg_vel_km.mean(axis=0)

array([ -4.3317449 , 208.11209189])